In [1]:
import serial
import time
import json

class ArduinoComm:
    def __init__(self, port, baudrate=9600):
        self.ser = serial.Serial(port, baudrate)
        time.sleep(2)  # Give some time for the connection to be established

    def send(self, command, data=None):
        msg = {"command": command, "data": data}
        encoded_msg = json.dumps(msg).encode()
        self.ser.write(encoded_msg)
        return self.receive_response()

    def receive_response(self):
        raw_data = self.ser.readline().decode().strip()
        response = json.loads(raw_data)
        return response

    def close(self):
        self.ser.close()
        

In [2]:
dataMaker = ArduinoComm("COM5")

In [4]:
response = dataMaker.send(command="getAll", data=None)
print(response)  # Should print the time (or an error if there's one)

{'therm1': 579, 'therm2': 538, 'soil1a': 200, 'soil1b': 292, 'soil2a': 251, 'soil2b': 566}


In [3]:
from pymongo import MongoClient

class MongoDBManager:

    def __init__(self, host='DESKTOP-M8400H1', port=27017, db_name='pf'):
        # Initialize MongoDB client
        self.client = MongoClient(host, port)
        self.db = self.client[db_name]

    def insert_one(self, collection_name, data):
        """Insert one record into a MongoDB collection."""
        collection = self.db[collection_name]
        collection.insert_one(data)

    def insert_many(self, collection_name, data_list):
        """Insert multiple records into a MongoDB collection."""
        collection = self.db[collection_name]
        collection.insert_many(data_list)

    def find(self, collection_name, query=None, projection=None):
        """Query data from a MongoDB collection."""
        collection = self.db[collection_name]
        return collection.find(query, projection)

    def update_one(self, collection_name, filter_query, update_data):
        """Update one record in a MongoDB collection."""
        collection = self.db[collection_name]
        collection.update_one(filter_query, {'$set': update_data})

    def delete_one(self, collection_name, query):
        """Delete one record from a MongoDB collection."""
        collection = self.db[collection_name]
        collection.delete_one(query)

    def close(self):
        """Close MongoDB connection."""
        self.client.close()

In [4]:
mongoSesh = MongoDBManager()

In [ ]:
while True:
    response = dataMaker.send(command="getAll", data=None)
    response["TS"] = time.time()
    mongoSesh.insert_one("dataMaker", response)
    time.sleep(1)